In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from webdriver_manager.firefox import GeckoDriverManager
from bs4 import BeautifulSoup
import csv
import pandas as pd
import time


options = Options()
options.headless = True
driver = webdriver.Firefox(firefox_binary="C:/Program Files/LibreWolf/librewolf.exe", options=options, executable_path=GeckoDriverManager().install())


df = pd.DataFrame(columns=['Number', 'Title', 'Year', 'Runtime', 'Genre', 'Rating', 'Budget', 'Votes', 'Metascore'])

url_template = "https://www.imdb.com/search/title/?genres=sci-fi&start={}&explore=title_type,genres&title_type=movie&ref_=adv_explore_rhs&language=en"
start = 1
movies = []

# sadece ilk 2 sayfanın verisini almak için döngüyü 2 kez çalıştırıyoruz
for start in range(1, 101, 50):
    url = url_template.format(start)
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    items = soup.select('.lister-item-content')
    if not items:
        break


    for item in soup.select('.lister-item-content'):
        number = item.select_one('.lister-item-header span').text.strip('()')
        title = item.select_one('.lister-item-header a').text.strip()
        runtime_element = item.select_one('.runtime')
        runtime = runtime_element.text.strip() if runtime_element else ''
        rating_element = item.select_one('.ratings-imdb-rating strong')
        rating = rating_element.text.strip() if rating_element else ''
        year = item.select_one('.lister-item-content .text-muted').text.strip()

        genre_element = item.select_one('.genre')
        genre = " - ".join(genre_element.text.strip().split(',')) if genre_element else ''

        # Metascore bilgisini al
        metascore_element = item.select_one('.metascore')
        metascore = metascore_element.text.strip() if metascore_element else ''


        link = item.select_one('.lister-item-header a')['href']
        driver.get(f"https://www.imdb.com{link}business")
        time.sleep(1)

        # Budget bilgisini al
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        votes_element = item.select_one('span[name="nv"]')
        if votes_element:
            votes = votes_element['data-value']
        else:
            votes = ''

        budget_elements = soup.findAll('span', {'class': 'ipc-metadata-list-item__list-content-item'})
        if len(budget_elements) > 2:
            budget = budget_elements[2].text.strip().split(' (')[0]
        else:
            budget = ''

        df = pd.concat([df, pd.DataFrame({'Number': number, 'Title': title, 'Year': year, 'Runtime': runtime, 'Genre': genre, 'Rating': rating, 'Budget': budget, 'Votes': votes, 'Metascore': metascore}, index=[0])], ignore_index=True)



driver.quit()

C:\Users\Sw4yt\AppData\Local\Temp\ipykernel_49136\257447954.py:11: DeprecationWarning: headless property is deprecated, instead use add_argument('-headless')
  options.headless = True
C:\Users\Sw4yt\AppData\Local\Temp\ipykernel_49136\257447954.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary="C:/Program Files/LibreWolf/librewolf.exe", options=options, executable_path=GeckoDriverManager().install())
C:\Users\Sw4yt\AppData\Local\Temp\ipykernel_49136\257447954.py:12: DeprecationWarning: firefox_binary has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(firefox_binary="C:/Program Files/LibreWolf/librewolf.exe", options=options, executable_path=GeckoDriverManager().install())


In [3]:
df

,Number,Title,Year,Runtime,Genre,Rating,Budget,Votes,Metascore
0,1.,The Hunger Games: The Ballad of Songbirds and ...,(2023),,Action - Adventure - Drama,,,,
1,2.,The Flash,(2023),,Action - Adventure - Fantasy,,1.90 : 1,,
2,3.,Guardians of the Galaxy Vol. 3,(2023),150 min,Action - Adventure - Comedy,8.4,"$35,000,000",29873,65
3,4.,Ant-Man and the Wasp: Quantumania,(2023),124 min,Action - Adventure - Comedy,6.2,"$200,000,000",125735,48
4,5.,Transformers: Rise of the Beasts,(2023),,Action - Adventure - Sci-Fi,,2.39 : 1,,
...,...,...,...,...,...,...,...,...,...
95,96.,Last Sentinel,(2023),117 min,Drama - Sci-Fi - Thriller,5.2,,1433,
96,97.,The Platform,(2019),94 min,Horror - Sci-Fi - Thriller,7.0,"$1,090,116",242678,73
97,98.,Galaxy Quest,(1999),102 min,Adventure - Comedy - Sci-Fi,7.4,"$45,000,000",168888,70
98,99.,The Thing,(1982),109 min,Horror - Mystery - Sci-Fi,8.2,"$15,000,000",436940,57


In [4]:
df.head(55)

,Number,Title,Year,Runtime,Genre,Rating,Budget,Votes,Metascore
0,1.,The Hunger Games: The Ballad of Songbirds and ...,(2023),,Action - Adventure - Drama,,,,
1,2.,The Flash,(2023),,Action - Adventure - Fantasy,,1.90 : 1,,
2,3.,Guardians of the Galaxy Vol. 3,(2023),150 min,Action - Adventure - Comedy,8.4,"$35,000,000",29873,65
3,4.,Ant-Man and the Wasp: Quantumania,(2023),124 min,Action - Adventure - Comedy,6.2,"$200,000,000",125735,48
4,5.,Transformers: Rise of the Beasts,(2023),,Action - Adventure - Sci-Fi,,2.39 : 1,,
5,6.,Avatar: The Way of Water,(2022),192 min,Action - Adventure - Fantasy,7.7,"$350,000,000",371857,67
6,7.,Mighty Morphin Power Rangers: Once & Always,(2023),55 min,Action - Adventure - Family,5.8,,4740,61
7,8.,Everything Everywhere All at Once,(2022),139 min,Action - Adventure - Comedy,7.9,"$14,300,000",438458,81
8,9.,65,(2023),93 min,Action - Adventure - Drama,5.4,"$32,041,403",39245,40
9,10.,Nope,(2022),130 min,Horror - Mystery - Sci-Fi,6.8,"$68,000,000",224223,77


In [5]:
df.info

<bound method DataFrame.info of    Number                                              Title    Year  Runtime   
0      1.  The Hunger Games: The Ballad of Songbirds and ...  (2023)           \
1      2.                                          The Flash  (2023)            
2      3.                     Guardians of the Galaxy Vol. 3  (2023)  150 min   
3      4.                  Ant-Man and the Wasp: Quantumania  (2023)  124 min   
4      5.                   Transformers: Rise of the Beasts  (2023)            
..    ...                                                ...     ...      ...   
95    96.                                      Last Sentinel  (2023)  117 min   
96    97.                                       The Platform  (2019)   94 min   
97    98.                                       Galaxy Quest  (1999)  102 min   
98    99.                                          The Thing  (1982)  109 min   
99   100.                                   Independence Day  (1996)  145 min

In [8]:
df.describe()

,Number,Title,Year,Runtime,Genre,Rating,Budget,Votes,Metascore
count,100,100,100,100,100,100,100,100,100
unique,100,100,41,57,31,38,64,89,44
top,1.,The Hunger Games: The Ballad of Songbirds and ...,(2023),,Action - Adventure - Sci-Fi,,,,
freq,1,1,17,12,20,12,14,12,18


In [6]:
df.describe().T

,count,unique,top,freq
Number,100,100,1.,1
Title,100,100,The Hunger Games: The Ballad of Songbirds and ...,1
Year,100,41,(2023),17
Runtime,100,57,,12
Genre,100,31,Action - Adventure - Sci-Fi,20
Rating,100,38,,12
Budget,100,64,,14
Votes,100,89,,12
Metascore,100,44,,18


In [12]:
df.isnull().sum()

Number       0
Title        0
Year         0
Runtime      0
Genre        0
Rating       0
Budget       0
Votes        0
Metascore    0
dtype: int64

In [13]:
df.isna().sum()

Number       0
Title        0
Year         0
Runtime      0
Genre        0
Rating       0
Budget       0
Votes        0
Metascore    0
dtype: int64

In [16]:
df.isin(['']).sum()

Number        0
Title         0
Year          1
Runtime      12
Genre         0
Rating       12
Budget       14
Votes        12
Metascore    18
dtype: int64